<a href="https://colab.research.google.com/github/shdb1/deep_learning/blob/main/Fashion_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **Install Tesnserflow**



In [ ]:
!pip install tensorflow-gpu==2.0.0-alpha0


     |████████████████████████████████| 332.1MB 24kB/s 
     |████████████████████████████████| 51kB 5.0MB/s 
     |████████████████████████████████| 3.0MB 35.7MB/s 
     |████████████████████████████████| 419kB 36.4MB/s 


 **1. Prepare dataset**

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf

#print(tfds.list_builders())
dataset, info = tfds.load('fashion_mnist', as_supervised = True, with_info = True)
dataset_test, dataset_train = dataset['test'], dataset['train']
print(info)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 



Failed to import TensorFlow. Please note that TensorFlow is not installed by default when you install TensorFlow Datasets. This is so that users can decide whether to install the GPU-enabled TensorFlow package. To use TensorFlow Datasets, please install the most recent version of TensorFlow, by following instructions at https://tensorflow.org/install.




ImportError: ignored

In [ ]:
def convert_types(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

batch_size = 32

dataset_train = dataset_train.map(convert_types).shuffle(10000).batch(batch_size)
dataset_test = dataset_test.map(convert_types).batch(batch_size)

 **2. Define model**


In [ ]:
from tensorflow.keras.layers import Conv2D, Flatten, Dense, BatchNormalization, Dropout, Activation, MaxPool2D, GlobalAveragePooling2D
from tensorflow.keras import Model

class CNNModel(Model):
    def __init__(self):
        super(CNNModel, self).__init__()
        drop_rate = 0.5
        
        self._layers = ([
            Conv2D(32, 3), # 28, 28, 1 -> 26, 26, 32
            BatchNormalization(),
            Activation(tf.nn.relu),
            Conv2D(64, 3), # 26, 26, 32 -> 24, 24, 64
            BatchNormalization(),
            Activation(tf.nn.relu),
            MaxPool2D(), # 24, 24, 64 -> 12, 12, 64
            Conv2D(128, 3), # 12, 12, 64 -> 10, 10, 128
            BatchNormalization(),
            Activation(tf.nn.relu),
            Conv2D(256, 3), # 10, 10, 128 -> 8, 8, 256
            BatchNormalization(),
            Activation(tf.nn.relu),
            MaxPool2D(), # 8, 8, 256 -> 4, 4, 256
            Flatten(), # 4, 4, 256 -> 4096
            Dense(256), # 4096 -> 256
            BatchNormalization(),
            Dropout(drop_rate),
            Activation(tf.nn.relu),
            Dense(10, activation = 'softmax') # 256 -> 10                        
        ])                
        
    def call(self, x):
        for layer in self._layers:
            x = layer(x)
        return x
       
    
model = CNNModel()

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

**3. Prepare training**

In [ ]:
train_loss = tf.keras.metrics.Mean(name = 'train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'train_accuracy')

test_loss = tf.keras.metrics.Mean(name = 'test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'test_accuracy')

In [ ]:
@tf.function
def train_step(image, label):
    with tf.GradientTape() as tape:
        predictions = model(image)
        loss = loss_object(label, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(label, predictions)
        
@tf.function
def test_step(image, label):
    predictions = model(image)
    loss = loss_object(label, predictions)
    
    test_loss(loss)
    test_accuracy(label, predictions)

**4. Train**


In [ ]:
import time

num_epoch = 10
start_time = time.time()

for epoch in range(num_epoch):
    for image, label in dataset_train:
        train_step(image, label)
        
    for test_image, test_label in dataset_test:
        test_step(test_image, test_label)
        
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}, spent_time: {} min'
    spent_time = time.time() - start_time
    print(template.format(epoch + 1, train_loss.result(), train_accuracy.result() * 100, test_loss.result(), test_accuracy.result() * 100, spent_time / 60))